# 1.单纯用python实现手写数字识别的神经网络算法

In [1]:
# 导入标准库
import random

In [2]:
# 导入numpy
import numpy as np

In [3]:
# 定义一个全链神经网络类
class Network(object):
    """
    定义__init__函数，函数的输入参数是一个列表，列表的每个
    值表示每层神经网络中的神经元个数。比如[2,3,1],表示第一
    层（输入层）有2个神经元,第二层有3个神经元，第三层有1个
    神经元。列表的维度表示网络的层数。
    """
    def __init__(self,sizes):
        
        #列表的长度作为层数传递给num_layers
        self.num_layers = len(sizes)
        
        #神经元的尺寸
        self.sizes = sizes
        
        #初始化偏置，第一层是输入层没有偏置，偏置矩阵列表长度等于
        #总层数-1 ，第二层开始到最后一层，每一层的偏置是一个
        # n行，1列的np矩阵。n是该层的神经元个数。
        self.biases = [np.random.randn(n,1) for n in sizes[1:]]
        
        # 初始化每层的权重，第一层是输入层，没有权重。第二层开始
        #每层的权重是一个np矩阵[n,m] n是当前层的神经元个数，m是
        #前一层的神经元个数。权重列表长度等于总层数-1
        
        self.weights = [np.random.randn(n,m) for 
                        n,m in zip(sizes[1:],sizes[:-1])]
                                                            
    # 定义一个feedforward方法来计算从第二层开始每一层神经元的输出值
    # 输出值=sigmoid（w.*a+b）a 是前一层是输出列表。w为当前层的权重
    # 矩阵，b为当前层的偏置矩阵
    def feedforward(self,a):
        
        #self.biases,self.weights中第一个矩阵是第二层的偏置跟权重矩阵
        for b,w in zip(self.biases,self.weights):
            
            #返回前一层输出值a输入当前层后，当前层是输出。用sigmoid
            #激活函数。
            return sigmoid(np.dot(w,a) + b)
    
    #定义随机梯度下降函数
    
    
    
    
        

In [4]:
a=[2,3,1]


In [14]:
a

[2, 3]

In [15]:
len(a
   )

2